In [2]:
import torch
import pandas as pd

C:\Users\rishi\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [169]:
from sklearn.model_selection import train_test_split

In [170]:
data = pd.read_csv('Data.csv')

In [171]:
x = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [172]:
x.head()

,AT,V,AP,RH
0,14.96,41.76,1024.07,73.17
1,25.18,62.96,1020.04,59.08
2,5.11,39.40,1012.16,92.14
3,20.86,57.32,1010.24,76.64
4,10.82,37.50,1009.23,96.62


In [173]:
y.head()

0    463.26
1    444.37
2    488.56
3    446.48
4    473.90
Name: PE, dtype: float64

In [174]:
data.head()

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


In [297]:
import numpy as np

In [298]:
def splits():
    return train_test_split(x, y, test_size=0.2, random_state=42)

In [299]:
def model():
    return torch.nn.Linear(4, 1)

In [300]:
def get_loss():
    return torch.nn.functional.mse_loss

In [301]:
def get_optimizer(model):
    return torch.optim.SGD(model.parameters(),lr=2e-7)

In [302]:
# x and y in pandas df
def get_data(x, y, batch_size):
    x_n = x.values.astype('float32')
    y_n = y.values.astype('float32')
    inputs = torch.from_numpy(x_n)
    targets = torch.from_numpy(y_n)
    train_ds = torch.utils.data.TensorDataset(inputs, targets)
    train_dl = torch.utils.data.DataLoader(train_ds, batch_size, shuffle=True)
    return train_dl

In [303]:
def akkuracy(models, data_x, data_y, pct_close):
    n_items = len(data_y)
    X = torch.Tensor(data_x)
    Y = torch.Tensor(data_y)  # actual as [102] Tensor

    oupt = models(X)       # predicted as [102,1] Tensor
    pred = oupt.view(n_items)  # predicted as [102]

    n_correct = torch.sum((torch.abs(pred - Y) < torch.abs(pct_close * Y)))
    acc = (n_correct.item() * 100.0 / n_items)  # scalar
    return acc 

In [310]:
def fit(epochs, loss, train_ds,x_test, y_test):
    models = model()
    opt = get_optimizer(models)
    final_acc = 0
    test_acc = 0
    for epoch in range(epochs):
        
        for xb, yb in train_ds:
            # Predictions
            predicts = models(xb)

            # Get errors
            losses = loss(predicts, yb.reshape(yb.shape[0], 1))
            
            # get back
            losses.backward()
            
            #Optimize
            opt.step()
            
            # Reset grad
            opt.zero_grad()
            
        if (epoch+1) % 200 == 0:
            final_acc+=akkuracy(models, xb, yb, 0.01)
            test_acc+=akkuracy(models, x_test, y_test, 0.0185)
            print("Epochs[{}/{}], Loss:{},Train Accuracy:{}, Test Accuracy:{}".format(epoch+1, 
                                                               epochs, losses.item(), 
                                                               akkuracy(models, xb, yb, 0.01),
                                                               akkuracy(models, x_test, y_test, 0.0185)))
    print(final_acc/(epochs//200))
    print(test_acc/(epochs//200))

In [311]:
x_train, x_test, y_train, y_test = splits()

In [312]:
x_test = torch.from_numpy(x_test.values.astype('float32'))
y_test = torch.from_numpy(y_test.values.astype('float32'))

In [313]:
train_dl = get_data(x_train, y_train, 100)

In [314]:
loss = get_loss()

In [315]:
fit(3000, loss, train_dl,x_test, y_test)

Epochs[200/3000], Loss:78.76124572753906,Train Accuracy:38.888888888888886, Test Accuracy:66.0919540229885
Epochs[400/3000], Loss:71.26722717285156,Train Accuracy:38.888888888888886, Test Accuracy:73.87669801462904
Epochs[600/3000], Loss:66.29273986816406,Train Accuracy:53.7037037037037, Test Accuracy:78.63113897596656
Epochs[800/3000], Loss:50.512115478515625,Train Accuracy:51.851851851851855, Test Accuracy:79.51933124346917
Epochs[1000/3000], Loss:40.68157196044922,Train Accuracy:61.111111111111114, Test Accuracy:81.03448275862068
Epochs[1200/3000], Loss:35.94533920288086,Train Accuracy:51.851851851851855, Test Accuracy:82.13166144200628
Epochs[1400/3000], Loss:55.811031341552734,Train Accuracy:46.2962962962963, Test Accuracy:82.86311389759666
Epochs[1600/3000], Loss:24.86627769470215,Train Accuracy:72.22222222222223, Test Accuracy:83.80355276907001
Epochs[1800/3000], Loss:30.943601608276367,Train Accuracy:59.25925925925926, Test Accuracy:84.43051201671891
Epochs[2000/3000], Loss:46.